# *To catch a protagonist* in DraCor

In the paper [*To Catch a Protagonist: Quantitative Domninance Relations in German-Language Drama (1730-1930)*](https://dh2018.adho.org/to-catch-a-protagonist-quantitative-dominance-relations-in-german-language-drama-1730-1930) ({cite:t}`fischer_catch_2018`) an algorithm is described, that allows to identify characters that are the quantitatively dominant characters of a play based on a set of network-based and count based measures:

> In order to systematically describe the extent of this deviation, we calculate eight values for each character of the 465 dramas of our corpus, three count-based measures (number of scenes a character appears in, number of speech acts, number of spoken words) and five network-related measures (degree, weighted degree, betweenness centrality, closeness centrality, eigenvector centrality). For each measurement a ranking is created. The rankings are then merged into two meta-rankings: one count-based and one network-based. The two meta-rankings are then combined into an overall ranking.

The original algorithm was implemented in the tool [Dramavis](https://github.com/lehkost/dramavis) by Christopher Kittel. *Dramavis* operates on XML ["zwischenformat"](https://dlina.github.io/Introducing-Our-Zwischenformat) files (cf. {cite:p}`kampkaspar_2015_zwischenformat`) created in the [DLINA](https://dlina.github.io) project. 

The following notebook[^catch_protagonist_notebook_source] adapts the code of the respective modules to work with data returned by the [DraCor API](https://dracor.org/doc/api). The aim is to be able to recreate the `*_chars.csv`-files that were used in the study. The data can be found in the [repository](https://github.com/dlina/catch-protagonist) on GitHub in the folder [`allmetrics`](https://github.com/dlina/catch-protagonist/tree/main/data/allmetrics).

[^catch_protagonist_notebook_source]: This notebook was originally published in the [dracor-notebooks](https://github.com/dracor-org/dracor-notebooks) GitHub repository and presented in the talk "To catch a protagonist … once again. An attempt to recreate a corpus-based study using Linked Data" at the [Distant Reading project's closing conference](https://www.distant-reading.net/events/conference-programme). For the ExploreCor Training School it has been adapted to work with major version v1 of the DraCor API.

The implementation will be tested with the play *Emilia Galotti*. The original algorithm operated on the corresponding [LINA](https://dlina.github.io/linas/88) and produced the file [`88_Emilia Galotti_chars.csv`](https://github.com/dlina/catch-protagonist/blob/main/data/allmetrics/88_Emilia%20Galotti_chars.csv) as output In DraCor the play can be accessed [here](https://dracor.org/ger/lessing-emilia-galotti).


## Step 1. Get the basic measures

We need to get the following basic measures on characters:

**Network measures**

* betweenness
* degree
* closeness
* ~~closeness corrected~~
* weighted degree
* eigenvector centrality

**count-based measures**

* frequency/appearances
* number of *speech acts*
* number of *words*

### Network and count-based metrics via Dracor API

The Python-Packages [`requests`](https://docs.python-requests.org/en/latest/) and the library [`json`](https://docs.python.org/3/library/json.html#module-json) will be used to query the API and parse the response:

In [34]:
# if not installed, uncomment the following line and run the cell:
# !pip install requests

In [35]:
import requests
import json

In [36]:
# set corpus and playname
corpusname = "cal"
playname = "amado-y-aborrecido"

In [37]:
# base url of the DraCor-API
api_base = "https://dracor.org/api/v1/"

To retrieve the network-data and speech-amounts data on single characters the function `/corpora/{corpusname}/plays/{playname}/characters` is used as follows:

In [38]:
# send a request to the endpoint and parse results
request_url = api_base + "corpora/" + corpusname + "/plays/" + playname + "/characters"
print(request_url)
r = requests.get(request_url)
character_data = json.loads(r.text)

https://dracor.org/api/v1/corpora/cal/plays/amado-y-aborrecido/characters


The API function returns data on the characters, including the network and count-based metrics:

In [39]:
character_data

[{'id': 'aurelio',
  'name': 'Aurelio',
  'isGroup': False,
  'gender': 'MALE',
  'numOfScenes': 17,
  'numOfSpeechActs': 42,
  'numOfWords': 1282,
  'degree': 18,
  'weightedDegree': 58,
  'closeness': 0.8214285714285714,
  'betweenness': 0.02156246257036771,
  'eigenvector': 0.2400623824759561},
 {'id': 'dante',
  'name': 'Dante',
  'isGroup': False,
  'gender': 'MALE',
  'numOfScenes': 41,
  'numOfSpeechActs': 184,
  'numOfWords': 4482,
  'degree': 23,
  'weightedDegree': 132,
  'closeness': 1,
  'betweenness': 0.07913822014612529,
  'eigenvector': 0.27133167504749744},
 {'id': 'flora',
  'name': 'Flora',
  'isGroup': False,
  'gender': 'FEMALE',
  'numOfScenes': 7,
  'numOfSpeechActs': 11,
  'numOfWords': 108,
  'degree': 17,
  'weightedDegree': 50,
  'closeness': 0.7931034482758621,
  'betweenness': 0.013163253084201703,
  'eigenvector': 0.23374859227091382},
 {'id': 'nise',
  'name': 'Nise',
  'isGroup': False,
  'gender': 'FEMALE',
  'numOfScenes': 7,
  'numOfSpeechActs': 10,
  

The data on the characters are in a dictionary:
```
{'betweenness': 0.24696969696969698,
  'closeness': 0.8,
  'degree': 9,
  'eigenvector': 0.44898463593218985,
  'gender': 'MALE',
  'id': 'marinelli',
  'isGroup': False,
  'name': 'Marinelli',
  'numOfScenes': 19,
  'numOfSpeechActs': 221,
  'numOfWords': 4343,
  'weightedDegree': 30}
```

We don't know anything about the network metrics of the whole play, though. If we want to retrieve this information, we would have to use the API function `/corpora/{corpusname}/plays/{playname}/metrics`, which would also tell us, if there are several sub-networks in a dictionary-field with the key `numConnectedComponents`. This could be relevant, because we can also calculate some network-metrics differently, e.g. the *closeness*.

## Preparation: Get the metrics and construct a pandas data frame

In the Dramavis implementation an object of the class `DramaAnalyzer` is created, which contains the information on characters in a pandas data frame. We will create the same data structure to be able to use the same methods for calculating means and ranking.

The rows in the [table](https://raw.githubusercontent.com/dlina/catch-protagonist/main/data/allmetrics/88_Emilia%20Galotti_chars.csv) are:

`name,betweenness,degree,closeness,closeness_corrected,strength,eigenvector_centrality,avg_distance,avg_distance_corrected,frequency,speech_acts,words,lines,chars ...`

We will not include all rows, but only the ones, that are relevant for the rankings:

`name`,`betweenness`,`degree`,`closeness`,~~closeness_corrected~~,`strength`,`eigenvector_centrality`,~~avg_distance,avg_distance_corrected~~,`frequency`,`speech_acts`,`words`,~~lines,chars~~ ...

following rows will be called differently to follow DraCor conventions of the [API output](https://dracor.org/api/corpora/ger/plays/lessing-emilia-galotti/characters):

* `name` → `id`; later this will be used to construct URIs
* `strength` → `weightedDegree`
* `eigenvector_centrality` → `eigenvector`
* `frequency` → `numOfScenes`
* `speech_acts` → `numOfSpeechActs`
* `words` → `numOfWords`



The package [`pandas`](https://pandas.pydata.org/) is used to handle the data as a dataframe. Therefore we need to import the package.

In [40]:
# if not installed, uncomment the following line and run the cell:
# !pip install pandas

In [41]:
import pandas as pd

First, we need to transform the parsed JSON API response to a list of lists, that is then turned into the data frame `df`.

In [44]:
# columns
cols = ["id","betweenness","degree","closeness","weightedDegree","eigenvector","numOfScenes","numOfSpeechActs","numOfWords"]

# prepare the data for the data frame
df_data = []
for character in character_data:
    row = []
    for key in cols:
        row.append(character[key])
    df_data.append(row)

# construct the data frame
df = pd.DataFrame(df_data, columns = cols)

#turn the column "id" to the index
df = df.set_index('id')
#output
df      

,betweenness,degree,closeness,weightedDegree,eigenvector,numOfScenes,numOfSpeechActs,numOfWords
id,,,,,,,,
aurelio,0.021562,18,0.821429,58,0.240062,17,42,1282
dante,0.079138,23,1.000000,132,0.271332,41,184,4482
flora,0.013163,17,0.793103,50,0.233749,7,11,108
nise,0.013163,17,0.793103,44,0.233749,7,10,96
irene,0.079138,23,1.000000,130,0.271332,35,128,3258
voz,0.000000,5,0.560976,5,0.077455,1,1,7
rey,0.021035,18,0.821429,75,0.240302,19,80,1967
aminta,0.058453,22,0.958333,136,0.266813,41,150,2471
lidoro,0.058453,22,0.958333,99,0.266813,33,117,2290


We can now query the data, e.g. output the values of a single character by requesting a row by its index value, which is the `id` of the character. 

In [45]:
# get the values of a single character
df.loc["flora"]

betweenness          0.013163
degree              17.000000
closeness            0.793103
weightedDegree      50.000000
eigenvector          0.233749
numOfScenes          7.000000
numOfSpeechActs     11.000000
numOfWords         108.000000
Name: flora, dtype: float64

## Step 2. Calculate the ranks

In Dramavis the function [`get_character_ranks`](https://github.com/lehkost/dramavis/blob/v0.4/dramalyzer.py#L282-L288) creates the rankings of the count-based and network-based measures. We will adapt this function to operate on the created data frame and rename the columns:

In [46]:
metrics_to_rank = ['degree', 'closeness', 'betweenness', 'weightedDegree', 'eigenvector', 'numOfScenes', 'numOfSpeechActs', 'numOfWords']
for metric in metrics_to_rank:
    df[metric + "_rank"] = df[metric].rank(method='min', ascending=False)
df   

,betweenness,degree,closeness,weightedDegree,eigenvector,numOfScenes,numOfSpeechActs,numOfWords,degree_rank,closeness_rank,betweenness_rank,weightedDegree_rank,eigenvector_rank,numOfScenes_rank,numOfSpeechActs_rank,numOfWords_rank
id,,,,,,,,,,,,,,,,
aurelio,0.021562,18,0.821429,58,0.240062,17,42,1282,6.0,6.0,6.0,7.0,8.0,7.0,7.0,7.0
dante,0.079138,23,1.000000,132,0.271332,41,184,4482,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0
flora,0.013163,17,0.793103,50,0.233749,7,11,108,9.0,9.0,9.0,8.0,9.0,8.0,11.0,11.0
nise,0.013163,17,0.793103,44,0.233749,7,10,96,9.0,9.0,9.0,9.0,9.0,8.0,12.0,12.0
irene,0.079138,23,1.000000,130,0.271332,35,128,3258,1.0,1.0,1.0,3.0,1.0,3.0,3.0,2.0
voz,0.000000,5,0.560976,5,0.077455,1,1,7,23.0,23.0,19.0,23.0,24.0,21.0,23.0,21.0
rey,0.021035,18,0.821429,75,0.240302,19,80,1967,6.0,6.0,7.0,5.0,7.0,5.0,6.0,6.0
aminta,0.058453,22,0.958333,136,0.266813,41,150,2471,3.0,3.0,3.0,1.0,3.0,1.0,2.0,3.0
lidoro,0.058453,22,0.958333,99,0.266813,33,117,2290,3.0,3.0,3.0,4.0,3.0,4.0,4.0,4.0


## Step 3. Rank on average and standard deviation of the individual rankings

In Dramavis the individual rankings are then used for the calculation of an average ranking and the standard deviation, which are then also ranked. This is done by the function [`get_centrality_ranks`](https://github.com/lehkost/dramavis/blob/v0.4/dramalyzer.py#L305-L310).

The following columns will be added to the data frame:

* (1) `centrality_rank_avg`: The average of all rankings
* (2) `centrality_rank_std`: Standard deviation of the rankings
* (3) `centrality_rank_avg_rank`: A ranking is created from the average of all rankings (1)
* (4) `centrality_rank_std_rank`: A ranking is created from the standard deviation of all rankings (2)

The following dramavis code is adapted accordingly to operate on the dataframe:

In [47]:
ranks = [c for c in df.columns if c.endswith("rank")]
df['centrality_rank_avg'] = df[ranks].sum(axis=1)/len(ranks)
df['centrality_rank_std'] = df[ranks].std(axis=1)/len(ranks)
for metric in ['centrality_rank_avg', 'centrality_rank_std']:
    df[metric + "_rank"] = df[metric].rank(method='min', ascending=True)
df

,betweenness,degree,closeness,weightedDegree,eigenvector,numOfScenes,numOfSpeechActs,numOfWords,degree_rank,closeness_rank,betweenness_rank,weightedDegree_rank,eigenvector_rank,numOfScenes_rank,numOfSpeechActs_rank,numOfWords_rank,centrality_rank_avg,centrality_rank_std,centrality_rank_avg_rank,centrality_rank_std_rank
id,,,,,,,,,,,,,,,,,,,,
aurelio,0.021562,18,0.821429,58,0.240062,17,42,1282,6.0,6.0,6.0,7.0,8.0,7.0,7.0,7.0,6.750,0.088388,7.0,4.0
dante,0.079138,23,1.000000,132,0.271332,41,184,4482,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.125,0.044194,1.0,1.0
flora,0.013163,17,0.793103,50,0.233749,7,11,108,9.0,9.0,9.0,8.0,9.0,8.0,11.0,11.0,9.250,0.145621,8.0,9.0
nise,0.013163,17,0.793103,44,0.233749,7,10,96,9.0,9.0,9.0,9.0,9.0,8.0,12.0,12.0,9.625,0.188243,10.0,11.0
irene,0.079138,23,1.000000,130,0.271332,35,128,3258,1.0,1.0,1.0,3.0,1.0,3.0,3.0,2.0,1.875,0.123879,2.0,7.0
voz,0.000000,5,0.560976,5,0.077455,1,1,7,23.0,23.0,19.0,23.0,24.0,21.0,23.0,21.0,22.125,0.205260,24.0,12.0
rey,0.021035,18,0.821429,75,0.240302,19,80,1967,6.0,6.0,7.0,5.0,7.0,5.0,6.0,6.0,6.000,0.094491,6.0,5.0
aminta,0.058453,22,0.958333,136,0.266813,41,150,2471,3.0,3.0,3.0,1.0,3.0,1.0,2.0,3.0,2.375,0.114516,3.0,6.0
lidoro,0.058453,22,0.958333,99,0.266813,33,117,2290,3.0,3.0,3.0,4.0,3.0,4.0,4.0,4.0,3.500,0.066815,4.0,3.0


Based on the calculation of `centrality_rank_avg_rank`, the "central" characters can be already queried as follows:

In [48]:
df[df["centrality_rank_avg_rank"] == 1].index.tolist()

['dante']

## Additional Step: Create Rankings and combined rankings of network-based and count-based metrics separately

In addition to a ranking that combines all metrics and rankings derived thereof, the function [`get_structural_ranking_measures`](https://github.com/lehkost/dramavis/blob/v0.4/dramalyzer.py#L318-L330) treats network-based and count-based values separately and only then aggregates them to a combined overall ranking.

The function adds the following rows to the data frame:

* (1) `avg_graph_rank`: a ranking based on the rankings of the network-values (*degree*, *closeness*, *betweenness*, *strength* or *weightedDegree* and *eigenvector centrality* or *eigenvector*)
* (2) `avg_content_rank`: a ranking based on the rankings of the count-based values (*frequency* or *numOfScenes*, *speech acts* and *words*)
* (3) `overall_avg`: the two rankings (1+2) are combined by calculating the mean
* (4) `overall_avg_rank`: based on the overall average (3) a ranking is created

The following code is adapted accordingly to operate on the dataframe. The ranking stability measures are not implemented here.

In [49]:
# renamed the columns to match the DraCor values here:
graph_ranks = ['degree_rank', 'closeness_rank', 'betweenness_rank', 'weightedDegree_rank', 'eigenvector_rank']
content_ranks = ['numOfScenes_rank', 'numOfSpeechActs_rank', 'numOfWords_rank']

avg_graph_rank = df[graph_ranks].mean(axis=1).rank(method='min')
avg_content_rank = df[content_ranks].mean(axis=1).rank(method='min')
df["avg_graph_rank"] = avg_graph_rank
df["avg_content_rank"] = avg_content_rank
df["overall_avg"] = df[["avg_graph_rank", "avg_content_rank"]].mean(axis=1)
df["overall_avg_rank"] = df["overall_avg"].rank(method='min')

# Option 1: Retrieve id values from the index directly:
id_values = list(df.index)
print("Retrieved IDs:", id_values)

# Option 2: Reset the index so that 'id' becomes a column
df = df.reset_index()  # Now 'id' is a column
print(df.head())


df.to_csv(f"{playname}_results.csv", index=False)


Retrieved IDs: ['aurelio', 'dante', 'flora', 'nise', 'irene', 'voz', 'rey', 'aminta', 'lidoro', 'uno', 'otro', 'todos', 'diana', 'venus', 'malandrín', 'unos', 'otros', 'clori', 'laura', 'coro-1', 'coro-2', 'música', 'criado', 'libio']
        id  betweenness  degree  closeness  weightedDegree  eigenvector  \
0  aurelio     0.021562      18   0.821429              58     0.240062   
1    dante     0.079138      23   1.000000             132     0.271332   
2    flora     0.013163      17   0.793103              50     0.233749   
3     nise     0.013163      17   0.793103              44     0.233749   
4    irene     0.079138      23   1.000000             130     0.271332   

   numOfScenes  numOfSpeechActs  numOfWords  degree_rank  ...  \
0           17               42        1282          6.0  ...   
1           41              184        4482          1.0  ...   
2            7               11         108          9.0  ...   
3            7               10          96          9

Based on the calculation of `overall_avg_rank`, the "central" characters can be queried as follows:

In [50]:
df[df["overall_avg_rank"] == 1].index.tolist()

[1]

It is also possible to output a table sorted by rank:

In [51]:
df["centrality_rank_avg_rank"].sort_values()

df.to_csv(f"{playname}_results_ranked.csv", index=False)